## Imports and credentials

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

with open('/Users/dusti/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

yelp_api

## Define search

In [3]:
LOCATION = 'Chicago, IL'
TERM = 'Burger'

In [4]:
JSON_FILE = "Data/results_in_progress_Chicago_burger.json"
JSON_FILE

'Data/results_in_progress_Chicago_burger.json'

In [5]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    

    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        
        os.makedirs(folder,exist_ok=True)
        
        
    
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Chicago_burger.json already exists.


## Determine how many results are already in the file

In [6]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    

n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 1000 previous results found.


## Determine how many pages needed

In [7]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

## Delete previous results file

In [8]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

## Re-run code with new function

In [9]:
import os
import json

def create_json_file(JSON_FILE, delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [10]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Chicago_burger.json not found. Saving empty list to new file.
- 0 previous results found.


415

In [11]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/415 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## Convert .json to df

In [12]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,xoi7Cw7FoknAx5p880RtWQ,au-cheval-chicago,Au Cheval,https://s3-media1.fl.yelpcdn.com/bphoto/ciecqv...,False,https://www.yelp.com/biz/au-cheval-chicago?adj...,8264,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 41.88466, 'longitude': -87.647668}","[delivery, pickup]",$$,"{'address1': '800 W Randolph St', 'address2': ...",+13129294580,(312) 929-4580,9071.234078
1,qxSpiDvad8awYWYZHNoPyQ,nadc-burger-chicago,NADC Burger,https://s3-media1.fl.yelpcdn.com/bphoto/1sRP-X...,False,https://www.yelp.com/biz/nadc-burger-chicago?a...,8,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 41.889635, 'longitude': -87.644829}",[],NaN,"{'address1': '415 N Milwaukee Ave', 'address2'...",,,9610.884910
2,vLNDwB7xaqE5OsmZ0Y3gng,salt-burgers-fries-chicago,Salt Burgers + Fries,https://s3-media2.fl.yelpcdn.com/bphoto/PwsZTc...,False,https://www.yelp.com/biz/salt-burgers-fries-ch...,114,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 41.91072, 'longitude': -87.67592}","[delivery, pickup]",$$,"{'address1': '1920 W North Ave', 'address2': N...",+13127865929,(312) 786-5929,9877.807949
3,9mhqcimD0CYvqxzfj_VXnQ,little-bad-wolf-chicago,Little Bad Wolf,https://s3-media3.fl.yelpcdn.com/bphoto/aN7fWm...,False,https://www.yelp.com/biz/little-bad-wolf-chica...,2007,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 41.983356, 'longitude': -87.669057}","[delivery, pickup]",$$,"{'address1': '1541 W Bryn Mawr Ave', 'address2...",+17739426399,(773) 942-6399,17601.237441
4,7Es7EbTy_s1btAdowLeiPQ,small-cheval-wicker-park-chicago,Small Cheval- Wicker Park,https://s3-media3.fl.yelpcdn.com/bphoto/oyVPVN...,False,https://www.yelp.com/biz/small-cheval-wicker-p...,1363,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 41.9128227233887, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '1732 N Milwaukee Ave', 'address2...",+13128373859,(312) 837-3859,9889.764457


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,n47jWHED87LNfosn1Ms5_g,pork-and-mindys-chicago-3,Pork & Mindys,https://s3-media1.fl.yelpcdn.com/bphoto/78y8wH...,False,https://www.yelp.com/biz/pork-and-mindys-chica...,2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.0,"{'latitude': 41.82474233, 'longitude': -87.616...","[pickup, delivery]",NaN,"{'address1': '3857 S Martin Luther King Dr', '...",,,9591.102053
996,NG-3BMzPBEf-t7b99Vo8UQ,the-fremont-bar-and-grill-westmont-2,The Fremont Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/RGMpN0...,False,https://www.yelp.com/biz/the-fremont-bar-and-g...,45,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 41.7744428337575, 'longitude': -8...",[delivery],$$,"{'address1': '1115 Fairview Ave', 'address2': ...",+16302978773,(630) 297-8773,22644.474792
997,fo4BhP2nim2vAY-o3fGGJA,bottoms-up-bar-and-lounge-schiller-park,Bottoms Up Bar & Lounge,https://s3-media2.fl.yelpcdn.com/bphoto/cCp8NZ...,False,https://www.yelp.com/biz/bottoms-up-bar-and-lo...,2,"[{'alias': 'bars', 'title': 'Bars'}]",4.5,"{'latitude': 41.955555, 'longitude': -87.869395}","[pickup, delivery]",NaN,"{'address1': '9718 W Irving Park Rd', 'address...",+12247812298,(224) 781-2298,17860.361243
998,C5UfBJEvJSG3WXXDHvBBRA,pizza-beer-and-jukebox-chicago,Pizza Beer & Jukebox,https://s3-media3.fl.yelpcdn.com/bphoto/qzI2v_...,False,https://www.yelp.com/biz/pizza-beer-and-jukebo...,178,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88593, 'longitude': -87.6496}","[pickup, delivery]",$$,"{'address1': '205 N Peoria St', 'address2': No...",+13124987080,(312) 498-7080,9038.385264
999,a3KEZk0jSKiK11LCw8MZeA,eddies-of-arlington-arlington-heights-2,Eddie's of Arlington,https://s3-media1.fl.yelpcdn.com/bphoto/4vYUW9...,False,https://www.yelp.com/biz/eddies-of-arlington-a...,126,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 42.08425, 'longitude': -87.982455}",[delivery],$,"{'address1': '10 E Northwest Hwy', 'address2':...",+18472221223,(847) 222-1223,34899.216430


## Check for duplicates

In [13]:
final_df.duplicated(subset='id').sum()

0

In [14]:
final_df.to_csv('Data/final_results_Chi_burgers.csv.gz', compression='gzip',index=False)